In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

import statsmodels.api as sm
import numpy as np
import itertools

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import resample

import xgboost as xgb
import shap

import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

rd = pd.read_excel('/sc/arion/projects/GENECAD/Robert/Drug_Phewas/RareDrug/Resources/combined_phecodex_list.xlsx')
cd = rd.loc[rd['Type'] == 'Common']

def cb(series):
    return series.apply(lambda x: 1 if x > 0 else 0)

## Import datasets

In [4]:
alldata = pd.read_pickle('./Final/ml_input.pkl')

alldata['xscore'] = 0.0
alldata.loc[alldata['phase'] == 0.5, 'xscore'] = 1*0.732*0.548*0.580*0.911
alldata.loc[alldata['phase'] == 1, 'xscore'] = 1*0.732*0.548*0.580
alldata.loc[alldata['phase'] == 2, 'xscore'] = 1*0.732*0.548
alldata.loc[alldata['phase'] == 3, 'xscore'] = 1*0.732
alldata.loc[alldata['phase'] == 4, 'xscore'] = 1

indata = alldata.loc[alldata['drug_gene'] == 1]
ind = indata.groupby('phecode')['indication'].max().reset_index()
ind = ind.loc[ind['indication'] == 1]
indata = indata.loc[indata['phecode'].isin(ind['phecode'])]
print(indata['phecode'].nunique())

#####

ot = ['clin_ot','hgmd','gwas_credible_sets','expression_atlas','impc','europepmc']
mantis = ['mantis']
cc = ['cc_common_max_p','cc_rare_max_p','cc_rare_burden_max_p','cc_ultrarare_max_p']

#####

apc = alldata.loc[alldata[ot+mantis+cc].max(axis=1) > 0][['id','gene','phecode','phecode_category','indication','phase','xscore']+ot+mantis+cc]


112


## Train models

In [5]:
run_shap = True
run_apc = True

if True:
    list1 = ['ot','mantis','cc','ot_mantis','ot_mantis_cc']
    list2 = [ot,mantis,cc,ot+mantis,ot+mantis+cc]

if False:
    list1 = ['ot_mantis_cc']
    list2 = [ot+mantis+cc]

for i,j in zip(list1, list2):
    print(i)
    
    X = indata[j]
    y = indata['xscore']
    ids = indata['id']

    holdout_predictions = []
    holdout_shap = pd.DataFrame()
    holdout_shap_interactions = []
    holdout_shap_ids = []
    
    apc_predictions = pd.DataFrame()
    
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_val_idx, holdout_idx) in enumerate(outer_cv.split(X), 1):
        print(f"Fold {fold}")
        
        X_train_val, X_holdout = X.iloc[train_val_idx], X.iloc[holdout_idx]
        y_train_val, y_holdout = y.iloc[train_val_idx], y.iloc[holdout_idx]
        ids_train_val, ids_holdout = ids.iloc[train_val_idx], ids.iloc[holdout_idx]

        apc_predictions_inner = []
        shap_inner = pd.DataFrame()
        
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
            print(f"  Inner Fold {inner_fold}")
            
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
            
            train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
            val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

            constraints = [1 if feature in [ot+mantis+cc] else 0
                           for feature in X.columns.tolist()]

            params = {'objective': 'reg:logistic', 'booster': 'gbtree',
                      'min_child_weight': 10, 'nthread': 16, 'monotone_constraints': tuple(constraints), 'seed': 0}

            evals = [(train_data, 'train'), (val_data, 'eval')]
            model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=10, evals=evals, verbose_eval=False)

            if i in ['ot_mantis','ot_mantis_cc']:
                model.save_model(f"./GPS/Main/Models/{i}_{fold}_{inner_fold}.json")
            
            y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
            auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
            auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
            print(auroc_hold, auprc_hold)

            fold_predictions = pd.DataFrame({
                'id': ids_holdout,
                'prediction': y_pred_hold
            })
            holdout_predictions.append(fold_predictions)
        
            if i in ['ot_mantis_cc'] and run_shap:
                if inner_fold in [1]:
                    X_holdout_background = X_holdout.sample(1000)        
                    explainer = shap.TreeExplainer(model, data=X_holdout_background, feature_perturbation='interventional', model_output='probability')
                    explanation = explainer(X_holdout)
                    explanation = pd.DataFrame(explanation.values, columns=X.columns)
                    explanation['id'] = ids_holdout.to_list()
                    shap_inner = pd.concat([shap_inner, explanation])

                    # Interactions only support raw values, not interventional
                    explainer = shap.TreeExplainer(model)
                    interactions = explainer.shap_interaction_values(X_holdout)
                    holdout_shap_interactions.append(interactions)
                    holdout_shap_ids.append(ids_holdout.to_list())

            if i in ['ot_mantis_cc'] and run_apc:
                apc_subset = apc.loc[~apc['id'].isin(ids_train_val)]
                y_pred_apc = model.predict(xgb.DMatrix(apc_subset[j], enable_categorical=True))
                
                fold_predictions = pd.DataFrame({
                    'id': apc_subset['id'],
                    'prediction': y_pred_apc
                })
                apc_predictions_inner.append(fold_predictions)            

        if i in ['ot_mantis_cc'] and run_shap:
            shap_inner = shap_inner.groupby('id').mean().reset_index()
            holdout_shap = pd.concat([holdout_shap, shap_inner])
            
        if i in ['ot_mantis_cc'] and run_apc:    
            apc_predictions_inner = pd.concat(apc_predictions_inner, ignore_index=True)
            apc_predictions_inner = apc_predictions_inner.groupby('id')['prediction'].mean().reset_index()
            apc_predictions = pd.concat([apc_predictions,apc_predictions_inner])

    holdout_predictions = pd.concat(holdout_predictions, ignore_index=True)
    holdout_predictions = holdout_predictions.groupby('id')['prediction'].mean().reset_index()
    all_predictions = holdout_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    all_predictions.to_pickle(f'./GPS/Main/Predictions/holdout_predictions_{i}.pkl')

    if i in ['ot_mantis_cc'] and run_shap:
        ##### Save SHAP predictions
        holdout_shap = holdout_shap.groupby('id').mean().reset_index()
        holdout_shap.to_pickle(f'./GPS/Main/SHAP/holdout_shap_{i}.pkl')

        holdout_shap_interactions = np.concatenate(holdout_shap_interactions, axis=0)
        holdout_shap_ids = [item for sublist in holdout_shap_ids for item in sublist]
        
        int_df = pd.DataFrame(np.mean(np.abs(holdout_shap_interactions), axis=0))
        int_df.columns = X.columns
        int_df.index = X.columns
        int_df.to_pickle(f'./GPS/Main/SHAP/holdout_shap_{i}_interactions.pkl')
        
        diagonal = pd.DataFrame(np.diagonal(holdout_shap_interactions, axis1=1, axis2=2), columns = X.columns)
        diagonal['id'] = holdout_shap_ids
        diagonal = diagonal.groupby('id').mean().reset_index()
        diagonal.to_pickle(f'./GPS/Main/SHAP/holdout_shap_{i}_diagonal.pkl')

    if i in ['ot_mantis_cc'] and run_apc:
        ##### Save predictions for all protein coding genes
        apc_predictions = apc_predictions.groupby('id')['prediction'].mean().reset_index()
        apc_predictions =  apc_predictions.merge(apc[['id','gene','phecode','indication','phase','xscore']])
        apc_predictions.to_pickle(f'./GPS/Main/Predictions/apc_predictions_{i}.pkl')
        apc_predictions = apc_predictions.loc[apc_predictions['id'].isin(alldata.loc[alldata['drug_gene'] == 1]['id'])]
        apc_predictions.to_pickle(f'./GPS/Main/Predictions/apc_druggable_predictions_{i}.pkl')


ot
Fold 1
  Inner Fold 1
0.6551299436653353 0.06500663415800555
  Inner Fold 2
0.6544321214766365 0.0650844342538659
  Inner Fold 3
0.65524101487449 0.06496470552275033
  Inner Fold 4
0.654356454287196 0.06511684774043813
  Inner Fold 5
0.6517906392828712 0.06369592814598879
Fold 2
  Inner Fold 1
0.6396440145814387 0.04179731224110485
  Inner Fold 2
0.6406875014899044 0.04271587494783864
  Inner Fold 3
0.6393857765237297 0.04109395134188484
  Inner Fold 4
0.6404473890975356 0.04244786139974
  Inner Fold 5
0.63965035946728 0.04181907263775882
Fold 3
  Inner Fold 1
0.6509822947014651 0.055670606937615524
  Inner Fold 2
0.6521145965937178 0.055829766654633656
  Inner Fold 3
0.6524886616751936 0.05611765622551551
  Inner Fold 4
0.6498208401627154 0.053814922710597644
  Inner Fold 5
0.6532544382715609 0.053280379694784774
Fold 4
  Inner Fold 1
0.6614867752768241 0.05269084179540928
  Inner Fold 2
0.6613573009982331 0.05422707746354116
  Inner Fold 3
0.661115247164254 0.05395952118202671
  I

 95%|=================== | 67448/71277 [00:15<00:00]       

  Inner Fold 2
0.7002152109499464 0.06963027721299443
  Inner Fold 3
0.7005053355190242 0.06975765148982382
  Inner Fold 4
0.707485941031607 0.07282406043030434
  Inner Fold 5
0.7048012089565232 0.07031599131803351
Fold 2
  Inner Fold 1
0.6820870328111264 0.050282018107187056


 98%|===================| 69848/71277 [00:23<00:00]        

  Inner Fold 2
0.6809729274436419 0.051060803251068744
  Inner Fold 3
0.6819638372810082 0.04896765138821816
  Inner Fold 4
0.6833786926453022 0.04972838025493233
  Inner Fold 5
0.6796985324411484 0.050628996627237514
Fold 3
  Inner Fold 1
0.7105293951464888 0.06290218311351045
  Inner Fold 2
0.7021709829934122 0.060901446506098224
  Inner Fold 3
0.7037423850845556 0.06256433416565234
  Inner Fold 4
0.7031139088175731 0.062210817652667866
  Inner Fold 5
0.7005503638500418 0.06072768324726157
Fold 4
  Inner Fold 1
0.7058417366145184 0.06373975851083566


 99%|===================| 70906/71277 [00:28<00:00]        

  Inner Fold 2
0.710430900569637 0.0655465471526605
  Inner Fold 3
0.7084152560185597 0.06530875036413497
  Inner Fold 4
0.7100620023254518 0.06963652051761679
  Inner Fold 5
0.7100211832782448 0.06745376007980433
Fold 5
  Inner Fold 1
0.6980575450337575 0.05426217770417094


 98%|===================| 69888/71276 [00:17<00:00]        

  Inner Fold 2
0.6940525959388125 0.052253307590172146
  Inner Fold 3
0.6982657092200335 0.055482336065509806
  Inner Fold 4
0.6928394066959237 0.05540160063214637
  Inner Fold 5
0.7016055005436928 0.05615516265979278


In [7]:
conds = ['ot','mantis','cc','ot_mantis','ot_mantis_cc']

for cond in conds:
    print('Holdout',cond)
    pdf = pd.read_pickle(f'./ML/Predictions/holdout_predictions_{cond}.pkl').reset_index(drop=True)
    auroc = roc_auc_score(pdf['indication'], pdf['prediction'])
    auprc = average_precision_score(pdf['indication'], pdf['prediction'])
    print(cond, auroc, auprc)
    

Holdout ot
ot 0.6195643597795137 0.03284980220496503
Holdout mantis
mantis 0.5619003127634152 0.020969205581835813
Holdout cc
cc 0.5627581834339868 0.01860106393715444
Holdout ot_mantis
ot_mantis 0.6562969190517017 0.03639632845885594
Holdout ot_mantis_cc
ot_mantis_cc 0.6809325956237589 0.04010750699732297


## Sensitivity 1: Gene holdout

In [8]:
if True:
    list1 = ['ot_mantis_cc']
    list2 = [ot+mantis+cc]

for i,j in zip(list1, list2):
    print(i)
    
    X = indata[j]
    y = indata['xscore']
    ids = indata['id']

    holdout_predictions = []
    
    genes = pd.Series(indata['gene'].unique()).sample(frac=1, random_state=42).reset_index(drop=True)
    gene_to_bucket = {g: ((idx % 5) + 1) for idx, g in genes.items()}
    gene_bucket = indata['gene'].map(gene_to_bucket)
    
    for fold in range(1, 6):
        print(f"Fold {fold}")
        holdout_mask = gene_bucket == fold
        
        X_train_val, X_holdout = X.loc[~holdout_mask], X.loc[holdout_mask]
        y_train_val, y_holdout = y.loc[~holdout_mask], y.loc[holdout_mask]
        ids_train_val, ids_holdout = ids.loc[~holdout_mask], ids.loc[holdout_mask]
        
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
            print(f"  Inner Fold {inner_fold}")
            
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
            
            train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
            val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

            constraints = [1 if feature in [ot+mantis+cc] else 0
                           for feature in X.columns.tolist()]

            params = {'objective': 'reg:logistic', 'booster': 'gbtree',
                      'min_child_weight': 10, 'nthread': 16, 'monotone_constraints': tuple(constraints), 'seed': 0}

            evals = [(train_data, 'train'), (val_data, 'eval')]
            model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=10, evals=evals, verbose_eval=False)

            y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
            auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
            auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
            print(auroc_hold, auprc_hold)

            fold_predictions = pd.DataFrame({
                'id': ids_holdout,
                'prediction': y_pred_hold,
                'bucket': fold
            })
            holdout_predictions.append(fold_predictions)
          
    holdout_predictions = pd.concat(holdout_predictions, ignore_index=True)
    holdout_predictions = holdout_predictions.groupby('id').agg({'prediction':'mean','bucket':'first'}).reset_index()
    holdout_predictions = holdout_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    holdout_predictions.to_pickle(f'./GPS/Sensitivity/gene_holdout_predictions_{i}.pkl')

ot_mantis_cc
Fold 1
  Inner Fold 1
0.6970742595508221 0.05906721261721384
  Inner Fold 2
0.6944652796572179 0.06015917729146039
  Inner Fold 3
0.7051553034071574 0.06523577739097586
  Inner Fold 4
0.7014306491796983 0.06070963050255811
  Inner Fold 5
0.7018173944825281 0.06423302587064228
Fold 2
  Inner Fold 1
0.6827772394474474 0.054241710672487536
  Inner Fold 2
0.6807361845680306 0.055846554205870355
  Inner Fold 3
0.6849102516452228 0.05872595522839838
  Inner Fold 4
0.6878327052223793 0.056913871945769794
  Inner Fold 5
0.6851863103462448 0.05425940798742889
Fold 3
  Inner Fold 1
0.6900882512887859 0.056490336855764614
  Inner Fold 2
0.686904089380186 0.054480408756149304
  Inner Fold 3
0.681799118150712 0.05484687075348548
  Inner Fold 4
0.6835814114064436 0.0533286385222546
  Inner Fold 5
0.6925303474781177 0.05711470502480834
Fold 4
  Inner Fold 1
0.6954399502284835 0.056035894624509815
  Inner Fold 2
0.692243161133764 0.05594792399130342
  Inner Fold 3
0.6950888646479338 0.053

## Sensitivity 2: Phecode category holdout

In [ ]:
apc.groupby('phecode_category')['indication'].sum()

In [9]:
if True:
    list1 = ['ot_mantis_cc']
    list2 = [ot+mantis+cc]

for i,j in zip(list1, list2):
    print(i)
    
    X = indata[j]
    y = indata['xscore']
    ids = indata['id']

    holdout_predictions = []
    
    cats = indata['phecode_category']
    groups = []
    for cat in cats.unique():
        groups.append(cats == cat)
    
    for fold, holdout_mask in enumerate(groups, 1):
        print(f"Fold {fold}")
        
        X_train_val, X_holdout = X.loc[~holdout_mask], X.loc[holdout_mask]
        y_train_val, y_holdout = y.loc[~holdout_mask], y.loc[holdout_mask]
        ids_train_val, ids_holdout = ids.loc[~holdout_mask], ids.loc[holdout_mask]
        
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
            print(f"  Inner Fold {inner_fold}")
            
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
            
            train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
            val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

            constraints = [1 if feature in [ot+mantis+cc] else 0
                           for feature in X.columns.tolist()]

            params = {'objective': 'reg:logistic', 'booster': 'gbtree',
                      'min_child_weight': 10, 'nthread': 16, 'monotone_constraints': tuple(constraints), 'seed': 0}

            evals = [(train_data, 'train'), (val_data, 'eval')]
            model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=10, evals=evals, verbose_eval=False)

            y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
            auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
            auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
            print(auroc_hold, auprc_hold)

            fold_predictions = pd.DataFrame({
                'id': ids_holdout,
                'prediction': y_pred_hold
            })
            holdout_predictions.append(fold_predictions)
          
    holdout_predictions = pd.concat(holdout_predictions, ignore_index=True)
    holdout_predictions = holdout_predictions.groupby('id')['prediction'].mean().reset_index()
    holdout_predictions = holdout_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore','phecode_category']])
    holdout_predictions.to_pickle(f'./GPS/Sensitivity/loo_holdout_predictions_{i}.pkl')

ot_mantis_cc
Fold 1
  Inner Fold 1
0.6632422232730579 0.04116645224106347
  Inner Fold 2
0.6592931219329818 0.042747410172437376
  Inner Fold 3
0.6574436293954351 0.04164476889616386
  Inner Fold 4
0.6504234390204823 0.04112818804400343
  Inner Fold 5
0.6671082781970266 0.04167193558063906
Fold 2
  Inner Fold 1
0.6554800507908536 0.060929557068554185
  Inner Fold 2
0.6625926347277685 0.06511087718016872
  Inner Fold 3
0.6648608562860068 0.06658674457573709
  Inner Fold 4
0.6550915264255839 0.062184799608826606
  Inner Fold 5
0.6578822641267097 0.06336766027392163
Fold 3
  Inner Fold 1
0.6062396427885786 0.026185782621188487
  Inner Fold 2
0.6027042884620774 0.026872167958496417
  Inner Fold 3
0.6129972309988194 0.02706386899005015
  Inner Fold 4
0.624152255555016 0.02637330908933245
  Inner Fold 5
0.6098807340072299 0.027239267298741945
Fold 4
  Inner Fold 1
0.7181143700690689 0.05124762362543913
  Inner Fold 2
0.7114387874579304 0.07003205214912613
  Inner Fold 3
0.7061754456076279 0.

## Sensitivity 3: Dataset holdout

In [10]:
if True:
    list1 = ['ot_mantis_cc']
    list2 = [ot+mantis+cc]

for i,j in zip(list1, list2):
    print(i)
    
    X = indata[j]
    y = indata['xscore']
    ids = indata['id']

    holdout_predictions = []
    
    minikel = indata['source'].astype(str).str.contains('Minikel', na=False)
    scenarios = [
        ((indata['indication'] == 0) | minikel, (indata['indication'] == 0) | (~minikel), 'train_Minikel_test_OT'),
        ((indata['indication'] == 0) | (~minikel), (indata['indication'] == 0) | minikel, 'train_OT_test_Minikel')
    ]
    
    for fold, (train_mask, test_mask, label) in enumerate(scenarios, 1):
        print(f"Fold {fold}")
        
        train_phe_keep = indata.loc[train_mask].groupby('phecode')['indication'].max()
        train_phe_keep = set(train_phe_keep.index[train_phe_keep == 1])
        test_phe_keep = indata.loc[test_mask].groupby('phecode')['indication'].max()
        test_phe_keep = set(test_phe_keep.index[test_phe_keep == 1])
        
        train_sel = train_mask & indata['phecode'].isin(train_phe_keep)
        test_sel = test_mask & indata['phecode'].isin(test_phe_keep)
        
        X_train_val, X_holdout = X.loc[train_sel], X.loc[test_sel]
        y_train_val, y_holdout = y.loc[train_sel], y.loc[test_sel]
        ids_train_val, ids_holdout = ids.loc[train_sel], ids.loc[test_sel]
        
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
            print(f"  Inner Fold {inner_fold}")
            
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
            
            train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
            val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

            constraints = [1 if feature in [ot+mantis+cc] else 0
                           for feature in X.columns.tolist()]

            params = {'objective': 'reg:logistic', 'booster': 'gbtree',
                      'min_child_weight': 10, 'nthread': 16, 'monotone_constraints': tuple(constraints), 'seed': 0}

            evals = [(train_data, 'train'), (val_data, 'eval')]
            model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=10, evals=evals, verbose_eval=False)

            y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
            auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
            auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
            print(auroc_hold, auprc_hold)

            fold_predictions = pd.DataFrame({
                'id': ids_holdout,
                'prediction': y_pred_hold,
                'scenario': label
            })
            holdout_predictions.append(fold_predictions)
          
    holdout_predictions = pd.concat(holdout_predictions, ignore_index=True)
    holdout_predictions = holdout_predictions.groupby(['id','scenario'])['prediction'].mean().reset_index()
    holdout_predictions = holdout_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    holdout_predictions.to_pickle(f'./GPS/Sensitivity/source_holdout_predictions_{i}.pkl')

ot_mantis_cc
Fold 1
  Inner Fold 1
0.6024100681762684 0.024830744778077504
  Inner Fold 2
0.6005384795252806 0.025286760842074592
  Inner Fold 3
0.6032559398303358 0.02610580227306305
  Inner Fold 4
0.6042781205673713 0.025389864602567494
  Inner Fold 5
0.598367237007068 0.02487006551927909
Fold 2
  Inner Fold 1
0.8140716424990391 0.03819107457284938
  Inner Fold 2
0.8058421866182497 0.03269980708198999
  Inner Fold 3
0.8131113035797531 0.03436658854938502
  Inner Fold 4
0.8152685739055049 0.038455758159043056
  Inner Fold 5
0.8183120821781126 0.03791973757877942
